In [1]:
import requests
import json
import os
import csv
import pandas as pd

In [2]:
api_key = "-o3YQPH9zXRxBs1zYaMp"
indicator_id = "ZSFH"
        #region_id = ["48310", "48892"]
region_id = "48310"

zillow_list = []

        #for region in region_id:
url = f"https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA?indicator_id={indicator_id}\
&region_id={region_id}&api_key={api_key}"
    
zillow = requests.get(url).json()

zillow_data = zillow['datatable']['data']
iterations_zillow = range(0,len(zillow_data))

for i in iterations_zillow:
    clean_zillow = zillow_data[i]
    zillow_list.append(clean_zillow)

zillow_df = pd.DataFrame(zillow_list, columns=['Indicator ID', 'Zip Code', 'Date', 'Average Value'])

    #add quarters to dataframe
zillow_df["Date"] = pd.to_datetime(zillow_df["Date"])
zillow_df["Quarter"] = zillow_df["Date"].dt.to_period('Q')

    #drop duplicate quarters to get end of quarter value and reset index
zillow_df = zillow_df.drop_duplicates(subset=['Quarter']).reset_index().drop(columns=["index",])


zillow_df.tail()

,Indicator ID,Zip Code,Date,Average Value,Quarter
103,ZSFH,48310,1997-03-31,102019.0,1997Q1
104,ZSFH,48310,1996-12-31,100316.0,1996Q4
105,ZSFH,48310,1996-09-30,99447.0,1996Q3
106,ZSFH,48310,1996-06-30,98922.0,1996Q2
107,ZSFH,48310,1996-03-31,98571.0,1996Q1


In [3]:
api_key = "-o3YQPH9zXRxBs1zYaMp"
start_date = "1996-03-01"
end_date = "2023-01-01"

unemployment_list = []

url = f"https://data.nasdaq.com/api/v3/datasets/FRED/NROUST.json?start_date={start_date}&end_date={end_date}&api_key={api_key}"
    
unemployment = requests.get(url).json()

unemployment_data = unemployment["dataset"]["data"]
iterations_unemployment = range(0,len(unemployment_data))

for i in iterations_unemployment:
    clean_unemployment = unemployment_data[i]
    unemployment_list.append(clean_unemployment)
    
unemployment_df = pd.DataFrame(unemployment_list, columns=['Date', 'Unemployment'])


unemployment_df["Date"] = pd.to_datetime(unemployment_df["Date"])
unemployment_df["Quarter"] = unemployment_df["Date"].dt.to_period('Q')

unemployment_df.head()

,Date,Unemployment,Quarter
0,2023-01-01,4.428064,2023Q1
1,2022-10-01,4.431760,2022Q4
2,2022-07-01,4.435784,2022Q3
3,2022-04-01,4.440214,2022Q2
4,2022-01-01,4.445137,2022Q1


In [4]:
    #change to merge on Quarter after figuring out how to change date
combined_df = pd.merge(zillow_df, unemployment_df, left_index=True, right_index=True)
combined_df = combined_df.drop(columns=["Quarter_y", "Date_y"]).rename(columns = {"Quarter_x":"Quarter", "Date_x":"Date"})
combined_df

,Indicator ID,Zip Code,Date,Average Value,Quarter,Unemployment
0,ZSFH,48310,2022-12-31,420386.0,2022Q4,4.428064
1,ZSFH,48310,2022-09-30,424230.0,2022Q3,4.431760
2,ZSFH,48310,2022-06-30,422575.0,2022Q2,4.435784
3,ZSFH,48310,2022-03-31,415626.0,2022Q1,4.440214
4,ZSFH,48310,2021-12-31,391604.0,2021Q4,4.445137
...,...,...,...,...,...,...
103,ZSFH,48310,1997-03-31,102019.0,1997Q1,5.332409
104,ZSFH,48310,1996-12-31,100316.0,1996Q4,5.341696
105,ZSFH,48310,1996-09-30,99447.0,1996Q3,5.350974
106,ZSFH,48310,1996-06-30,98922.0,1996Q2,5.360292
